In [21]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import glob as glob

%matplotlib inline
import sys
sys.path.append('/Users/eisaadil/.local/lib/python3.6/site-packages/tensorflow/models/research/object_detection') # ~/tensorflow/models/research/object_detection
from utils import label_map_util
from utils import visualization_utils as vis_util

import matplotlib.pyplot as plt

import cv2
import socket
import time

MODEL_NAME = 'ssd'

# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('scripts', 'label_map.pbtxt')

NUM_CLASSES = 78

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [22]:
robot_ip_address = "192.168.1.1"
robot_port = 2001
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((robot_ip_address, robot_port))

def left(speed):
    s.send(bytes([115,speed,0,0]))
    s.send(bytes([5])) #STOP
    time.sleep(0.7)
    s.send(bytes([0])) #LEFT
    print("Turn Left")

def right(speed):
    s.send(bytes([115,speed,0,0]))
    s.send(bytes([5])) #STOP
    time.sleep(0.7)
    s.send(bytes([2])) #LEFT
    time.sleep(0.7)
    forward(speed)
    print("Turn Right")
    
def stop():
    print("Stop")
    s.send(bytes([5]))
    
def sixty(speed):
    print("60km/h")
    s.send(bytes([115,speed,0,0]))
    
def forty(speed):
    print("40km/h")
    s.send(bytes([115,speed,0,0]))    
    
def forward(speed):
    s.send(bytes([115,speed,0,0]))
    s.send(bytes([1]))
    print("Go Forward")  


In [23]:
cap = cv2.VideoCapture("http://192.168.1.1:8080/?action=stream")
IMAGE_SIZE = (20, 20)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        if (cap.isOpened()== False): 
              print("Error opening video stream or file")
        forward(60)     
        ret = True
        while (ret):
            start_time = time.time()
            ret, image_np = cap.read() 

            # Image Shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis = 0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections], feed_dict = {
                image_tensor: image_np_expanded
              })
            
            if scores[0][0]>0.5:
                if (classes[0][0]==15): #LEFT
                    left(150)
                elif (classes[0][0]==40): #RIGHT
                    right(150)
                elif (classes[0][0]==76): #STOP
                    stop()
                elif (classes[0][0]==16): #60 SPEED
                    sixty(100)
                elif (classes[0][0]==67): #40 SPEED (DEFAULT) 
                    forty(60)
                elif (classes[0][0]==36):   
                    forward(60)  
            
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates = True,
              line_thickness = 8)
            
#             plt.figure(figsize = IMAGE_SIZE)
#             plt.imshow(image_np)
            cv2.imshow('image',cv2.resize(image_np,(1280,960)))
            print("--- %s seconds ---" % (time.time() - start_time))
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                cap.release()
                break

Go Forward
--- 8.798331022262573 seconds ---
--- 0.22805118560791016 seconds ---
--- 0.2725520133972168 seconds ---
--- 0.21209979057312012 seconds ---
--- 0.2305006980895996 seconds ---
--- 0.2286982536315918 seconds ---
--- 0.2112729549407959 seconds ---
--- 0.21019792556762695 seconds ---
--- 0.21388006210327148 seconds ---
--- 0.21047592163085938 seconds ---
--- 0.22667694091796875 seconds ---
--- 0.2122328281402588 seconds ---
--- 0.21681427955627441 seconds ---
--- 0.2140350341796875 seconds ---
--- 0.21024298667907715 seconds ---
--- 0.21548199653625488 seconds ---
--- 0.22582769393920898 seconds ---
--- 0.20891904830932617 seconds ---
--- 0.20051026344299316 seconds ---
--- 0.218214750289917 seconds ---
Stop
--- 0.20381617546081543 seconds ---
Stop
--- 0.22373175621032715 seconds ---
Stop
--- 0.201859712600708 seconds ---
Stop
--- 0.21648383140563965 seconds ---
Stop
--- 0.24512124061584473 seconds ---
Stop
--- 0.2051389217376709 seconds ---
--- 0.21579384803771973 seconds ---


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
forward(60) 

In [ ]:
stop()

In [ ]:
left(150)

In [ ]:
right(150)

In [ ]:
sixty(100)

In [ ]:
forty(60)